In [1]:
import yfinance as yahoo
import pandas as pd
import numpy as np
import scipy.optimize as sco
import copy
from scipy import stats
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
# import investment to track performance during 2020
portfolio = pd.read_excel('portfolioBacktest.xlsx')

portfolio

,Unnamed: 0,SharpeRatio,MinVaR,SortinoRatio,Sharpe,Benchmark,ComponentVaR
0,AAPL,0.0000,0.0043,0.067989,0.069385,0.04,0.045973
1,ADBE,0.1340,0.0029,0.106483,0.107081,0.04,0.109495
2,AMD,0.1124,0.0001,0.109923,0.122687,0.04,0.143737
3,AMZN,0.1500,0.0014,0.127529,0.121341,0.04,0.140599
4,BA,0.0874,0.0003,0.061965,0.067033,0.04,0.042908
5,BABA,0.0000,0.0650,0.000000,0.000000,0.04,0.000000
6,BAC,0.0000,0.0021,0.000000,0.000000,0.04,0.000000
7,C,0.0000,0.1380,0.000000,0.000000,0.04,0.000000
8,CMCSA,0.0000,0.0532,0.000000,0.000000,0.04,0.000000
9,CSCO,0.0000,0.0548,0.000000,0.000000,0.04,0.000000


In [3]:
# Remove 0's of ComponentVaR, for better visualization
portfolio = portfolio[portfolio.ComponentVaR!=0].dropna()
portfolio

,Unnamed: 0,SharpeRatio,MinVaR,SortinoRatio,Sharpe,Benchmark,ComponentVaR
0,AAPL,0.0000,0.0043,0.067989,0.069385,0.04,0.045973
1,ADBE,0.1340,0.0029,0.106483,0.107081,0.04,0.109495
2,AMD,0.1124,0.0001,0.109923,0.122687,0.04,0.143737
3,AMZN,0.1500,0.0014,0.127529,0.121341,0.04,0.140599
4,BA,0.0874,0.0003,0.061965,0.067033,0.04,0.042908
15,MA,0.0415,0.0028,0.108071,0.098558,0.04,0.092757
16,MSFT,0.0547,0.0022,0.112323,0.097381,0.04,0.090556
17,NFLX,0.0927,0.0027,0.092824,0.101962,0.04,0.099276
18,NVDA,0.1500,0.0002,0.124430,0.135191,0.04,0.174526
22,V,0.0000,0.0039,0.088463,0.079381,0.04,0.060173


In [4]:
weights = portfolio['ComponentVaR'].values
print(weights)

[0.04597268 0.10949534 0.1437367  0.14059917 0.04290822 0.09275743
 0.09055592 0.09927598 0.17452584 0.06017271]


In [5]:
# Set Initial quantity to simulate first Buy-&-Hold Strategy, with nominal fixed
# Capital requested to allocate investment
initial_capital = float(input("How much will you invest? "))

In [6]:
def AdjustRisk(portfolio, comienzo, final):
  """Provide the stock list of your portfolio
     to update risk by Component-Value-at-Risk"""  
  listado = list(portfolio.index.values)
  weights = list(portfolio.weights.values)
  df = yahoo.download(listado, start=comienzo, end=final,interval="60m")['Adj Close'].fillna(method='ffill')
  data = df 
  returns = data.pct_change()
  correlation = returns.corr() # correlation
  covariance = returns.cov()  # covariance
  instruments = pd.DataFrame(index= data.columns)
  instruments['weigths'] = 1/len(instruments.index) # secure allocation is equal 1
  instruments['deltas'] = (instruments.weigths * correlation).sum() # deltas as elasticity of the assets
  instruments['Stdev'] = returns.std()
  instruments['stress'] = (instruments.deltas * instruments.Stdev) * 3 # stress applied at 4 deviations
  instruments['portfolio_stress'] = instruments.stress.sum() # the stress of the portfolio
  risk = pd.DataFrame(index=data.columns)
  risk['numerator'] = (instruments.deltas.multiply(covariance)).sum()
  risk['denominator'] = data.pct_change().std() * (-2.365)
  risk['GradVaR'] = -risk.numerator / risk.denominator
  risk['CVaRj'] = risk.GradVaR * instruments.deltas # Component VaR of the Risk Factors j
  risk['thetai'] = (risk.CVaRj * correlation).sum() # Theta i of the instruments
  risk['CVaRi'] = risk.thetai * (1/len(data.columns)) # Component VaR of the Instruments i
  risk['totalCVaRi'] = risk.CVaRi.sum() #total CVaR of the portfolio
  risk['CVaRattribution'] = risk.CVaRi / risk.totalCVaRi # risk allocation by instrument in the portfolio
  riskadj = pd.DataFrame(index=data.columns)
  riskadj['base'] = instruments['weigths'].values
  riskadj['CVaRattribution'] = risk.CVaRattribution.sort_values(axis=0,ascending=False)
  riskadj['new'] = portfolio['weights'].values  # Choosing the option with the highest return
  riskadj['condition'] = (riskadj.base / riskadj.CVaRattribution)
  riskadj['newrisk'] = (riskadj.new / riskadj.CVaRattribution)
  riskadj['differences'] = (riskadj.newrisk - riskadj.condition)  # apply this result as a percentage to multiply new weights
  riskadj['adjustments'] = (riskadj.newrisk - riskadj.condition) / riskadj.condition #ALARM if its negative sum up the difference, 
                                              #if it is positive rest it, you need to have 0
  riskadj['suggested'] = riskadj.new * (1 + riskadj.adjustments)   
  riskadj['tototal'] = riskadj.suggested.sum()
  riskadj['MinCVaR'] = riskadj.suggested / riskadj.tototal
  result = pd.DataFrame(riskadj['MinCVaR'].values,columns=['MinCVaR'],index=data.columns)
  result[result.MinCVaR>=0.12] = 0.12
  result['MinCVaR'] = result['MinCVaR'] / sum(result['MinCVaR'])
  result['lastPrice'] = (data.tail(1).T.values)
  return result

In [7]:
# Cartera CVaR
stocks = list(portfolio.iloc[:,0].values)

start, end = dt.date(2019,12,25), dt.date(2019,12,31)
df =yahoo.download(stocks,start,end)["Adj Close"].fillna(method="ffill")
best = pd.DataFrame(index=portfolio.iloc[:,0].values)
best = pd.DataFrame(index=df.columns)
best['capital'] = initial_capital
best['price'] = df.tail(1).T.values
best['weights'] = portfolio['ComponentVaR'].values 
best['cash'] = (best['capital'] * best['weights'])
best['nominal'] =  best['cash'] // best['price'] 
best['invested'] = best['price'] * best['nominal']
best['percentage'] = best['invested'] / sum(best['invested'])
best['total'] = sum(best['invested'])
best['liquid'] = best['capital'] - best['total']
best = best[best.nominal!=0].dropna() # remove all stocks that you do not invest in
#cvarport = cvarport[cvarport!=0].dropna()
best


[*********************100%***********************]  10 of 10 completed


,capital,price,weights,cash,nominal,invested,percentage,total,liquid
AAPL,10000000.0,72.255997,0.045973,4.597268e+05,6362.0,4.596927e+05,0.045976,9.998480e+06,1520.131897
ADBE,10000000.0,328.339996,0.109495,1.094953e+06,3334.0,1.094686e+06,0.109485,9.998480e+06,1520.131897
AMD,10000000.0,45.520000,0.143737,1.437367e+06,31576.0,1.437340e+06,0.143756,9.998480e+06,1520.131897
AMZN,10000000.0,1846.890015,0.140599,1.405992e+06,761.0,1.405483e+06,0.140570,9.998480e+06,1520.131897
BA,10000000.0,324.469513,0.042908,4.290822e+05,1322.0,4.289487e+05,0.042901,9.998480e+06,1520.131897
MA,10000000.0,295.748596,0.092757,9.275743e+05,3136.0,9.274676e+05,0.092761,9.998480e+06,1520.131897
MSFT,10000000.0,155.938049,0.090556,9.055592e+05,5807.0,9.055323e+05,0.090567,9.998480e+06,1520.131897
NFLX,10000000.0,323.309998,0.099276,9.927598e+05,3070.0,9.925617e+05,0.099271,9.998480e+06,1520.131897
NVDA,10000000.0,231.937271,0.174526,1.745258e+06,7524.0,1.745096e+06,0.174536,9.998480e+06,1520.131897
V,10000000.0,186.680908,0.060173,6.017271e+05,3223.0,6.016726e+05,0.060176,9.998480e+06,1520.131897


# Rebalance Monthly

In [8]:
enero, febrero = dt.date(2020,1,1), dt.date(2020,2,1)
stocks = list(best.index)
data = yahoo.download(stocks,enero,febrero)['Adj Close'].fillna(method="ffill")

[*********************100%***********************]  10 of 10 completed


In [9]:
cartera = pd.DataFrame(best)
portfolio = pd.DataFrame(index=cartera.index)
portfolio['nominal'] = cartera['nominal'].values
portfolio['pricePaid'] = cartera['price'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = cartera['liquid'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = (yahoo.download(stocks, enero, febrero)['Adj Close'].fillna(method='ffill')).tail(1).T
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['weights'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
enero = copy.copy(portfolio)
portfolio

[*********************100%***********************]  10 of 10 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest
AAPL,6362.0,72.255997,0.045976,9.998480e+06,1520.131897,76.714989,1.052619e+07,1.052779,1.061711,6309.0,-53.0,0.045984,1.052524e+07,2468.147667
ADBE,3334.0,328.339996,0.109485,9.998480e+06,1520.131897,351.140015,1.052619e+07,1.052779,1.069440,3282.0,-52.0,0.109493,1.052524e+07,2468.147667
AMD,31576.0,45.520000,0.143756,9.998480e+06,1520.131897,47.000000,1.052619e+07,1.052779,1.032513,32200.0,624.0,0.143788,1.052524e+07,2468.147667
AMZN,761.0,1846.890015,0.140570,9.998480e+06,1520.131897,2008.719971,1.052619e+07,1.052779,1.087623,736.0,-25.0,0.140464,1.052524e+07,2468.147667
BA,1322.0,324.469513,0.042901,9.998480e+06,1520.131897,316.387573,1.052619e+07,1.052779,0.975092,1427.0,105.0,0.042895,1.052524e+07,2468.147667
MA,3136.0,295.748596,0.092761,9.998480e+06,1520.131897,314.266693,1.052619e+07,1.052779,1.062614,3107.0,-29.0,0.092770,1.052524e+07,2468.147667
MSFT,5807.0,155.938049,0.090567,9.998480e+06,1520.131897,168.445557,1.052619e+07,1.052779,1.080208,5660.0,-147.0,0.090582,1.052524e+07,2468.147667
NFLX,3070.0,323.309998,0.099271,9.998480e+06,1520.131897,345.089996,1.052619e+07,1.052779,1.067366,3028.0,-42.0,0.099279,1.052524e+07,2468.147667
NVDA,7524.0,231.937271,0.174536,9.998480e+06,1520.131897,236.040497,1.052619e+07,1.052779,1.017691,7784.0,260.0,0.174565,1.052524e+07,2468.147667
V,3223.0,186.680908,0.060176,9.998480e+06,1520.131897,197.752747,1.052619e+07,1.052779,1.059309,3203.0,-20.0,0.060179,1.052524e+07,2468.147667


In [10]:
comienzo = febrero - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, febrero)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
febrero = copy.copy(portfolio)
portfolio

[*********************100%***********************]  10 of 10 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
AAPL,6309.0,76.714989,0.045984,1.052524e+07,2468.147667,309.390015,1.201541e+07,1.141581,4.032980,928.0,-5381.0,0.023896,1.201495e+07,2936.030449,0.023916
ADBE,3282.0,351.140015,0.109493,1.052524e+07,2468.147667,351.140015,1.201541e+07,1.141581,1.000000,4641.0,1359.0,0.135634,1.201495e+07,2936.030449,0.135623
AMD,32200.0,47.000000,0.143788,1.052524e+07,2468.147667,46.970001,1.201541e+07,1.141581,0.999362,40683.0,8483.0,0.159042,1.201495e+07,2936.030449,0.159004
AMZN,736.0,2008.719971,0.140464,1.052524e+07,2468.147667,2007.760010,1.201541e+07,1.141581,0.999522,951.0,215.0,0.158917,1.201495e+07,2936.030449,0.159004
BA,1427.0,316.387573,0.042895,1.052524e+07,2468.147667,318.010010,1.201541e+07,1.141581,1.005128,786.0,-641.0,0.020804,1.201495e+07,2936.030449,0.020824
MA,3107.0,314.266693,0.092770,1.052524e+07,2468.147667,315.839996,1.201541e+07,1.141581,1.005006,3704.0,597.0,0.097368,1.201495e+07,2936.030449,0.097354
MSFT,5660.0,168.445557,0.090582,1.052524e+07,2468.147667,170.169998,1.201541e+07,1.141581,1.010237,6554.0,894.0,0.092826,1.201495e+07,2936.030449,0.092803
NFLX,3028.0,345.089996,0.099279,1.052524e+07,2468.147667,345.049988,1.201541e+07,1.141581,0.999884,3883.0,855.0,0.111514,1.201495e+07,2936.030449,0.111499
NVDA,7784.0,236.040497,0.174565,1.052524e+07,2468.147667,236.449997,1.201541e+07,1.141581,1.001735,8081.0,297.0,0.159031,1.201495e+07,2936.030449,0.159004
V,3203.0,197.752747,0.060179,1.052524e+07,2468.147667,198.960007,1.201541e+07,1.141581,1.006105,2474.0,-729.0,0.040968,1.201495e+07,2936.030449,0.040971


# Re-escribimos columna Cantidad Nominal, aplicando el cambio de la columna ajuste y buscamos nueva data del otro mes

In [11]:
marzo = dt.date(2020,3,1)
comienzo = marzo - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, marzo)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
marzo = copy.copy(portfolio)
portfolio

[*********************100%***********************]  10 of 10 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
AAPL,928.0,309.390015,0.023896,1.201495e+07,2936.030449,274.640015,1.192651e+07,0.99264,0.887682,1039.0,111.0,0.023926,1.192651e+07,2941.061638,0.023922
ADBE,4641.0,351.140015,0.135634,1.201495e+07,2936.030449,345.459991,1.192651e+07,0.99264,0.983824,4683.0,42.0,0.135647,1.192651e+07,2941.061638,0.135631
AMD,40683.0,46.970001,0.159042,1.201495e+07,2936.030449,45.470001,1.192651e+07,0.99264,0.968065,41711.0,1028.0,0.159024,1.192651e+07,2941.061638,0.158987
AMZN,951.0,2007.760010,0.158917,1.201495e+07,2936.030449,1887.439941,1.192651e+07,0.99264,0.940072,1004.0,53.0,0.158889,1.192651e+07,2941.061638,0.158987
BA,786.0,318.010010,0.020804,1.201495e+07,2936.030449,275.220001,1.192651e+07,0.99264,0.865444,902.0,116.0,0.020815,1.192651e+07,2941.061638,0.020817
MA,3704.0,315.839996,0.097368,1.201495e+07,2936.030449,290.380005,1.192651e+07,0.99264,0.919390,3999.0,295.0,0.097365,1.192651e+07,2941.061638,0.097364
MSFT,6554.0,170.169998,0.092826,1.201495e+07,2936.030449,162.110001,1.192651e+07,0.99264,0.952636,6830.0,276.0,0.092836,1.192651e+07,2941.061638,0.092827
NFLX,3883.0,345.049988,0.111514,1.201495e+07,2936.030449,369.649994,1.192651e+07,0.99264,1.071294,3598.0,-285.0,0.111516,1.192651e+07,2941.061638,0.111506
NVDA,8081.0,236.449997,0.159031,1.201495e+07,2936.030449,270.109985,1.192651e+07,0.99264,1.142356,7021.0,-1060.0,0.159011,1.192651e+07,2941.061638,0.158987
V,2474.0,198.960007,0.040968,1.201495e+07,2936.030449,182.330002,1.192651e+07,0.99264,0.916415,2680.0,206.0,0.040971,1.192651e+07,2941.061638,0.040971


In [12]:
abril = dt.date(2020,4,1)
comienzo = abril - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, abril)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
abril = copy.copy(portfolio)
portfolio

[*********************100%***********************]  10 of 10 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
AAPL,1039.0,274.640015,0.023926,1.192651e+07,2941.061638,254.300003,1.141906e+07,0.957453,0.925939,271.0,-768.0,0.006034,1.142046e+07,1548.694088,0.006050
ADBE,4683.0,345.459991,0.135647,1.192651e+07,2941.061638,318.390015,1.141906e+07,0.957453,0.921641,5812.0,1129.0,0.162032,1.142046e+07,1548.694088,0.162029
AMD,41711.0,45.470001,0.159024,1.192651e+07,2941.061638,45.439999,1.141906e+07,0.957453,0.999340,40728.0,-983.0,0.162050,1.142046e+07,1548.694088,0.162029
AMZN,1004.0,1887.439941,0.158889,1.192651e+07,2941.061638,1949.790039,1.141906e+07,0.957453,1.033034,949.0,-55.0,0.162021,1.142046e+07,1548.694088,0.162029
BA,902.0,275.220001,0.020815,1.192651e+07,2941.061638,149.119995,1.141906e+07,0.957453,0.541821,351.0,-551.0,0.004583,1.142046e+07,1548.694088,0.004585
MA,3999.0,290.380005,0.097365,1.192651e+07,2941.061638,241.539993,1.141906e+07,0.957453,0.831807,4749.0,750.0,0.100440,1.142046e+07,1548.694088,0.100439
MSFT,6830.0,162.110001,0.092836,1.192651e+07,2941.061638,157.639999,1.141906e+07,0.957453,0.972426,6614.0,-216.0,0.091295,1.142046e+07,1548.694088,0.091286
NFLX,3598.0,369.649994,0.111516,1.192651e+07,2941.061638,375.549988,1.141906e+07,0.957453,1.015961,4006.0,408.0,0.131733,1.142046e+07,1548.694088,0.131742
NVDA,7021.0,270.109985,0.159011,1.192651e+07,2941.061638,263.609985,1.141906e+07,0.957453,0.975936,7020.0,-1.0,0.162037,1.142046e+07,1548.694088,0.162029
V,2680.0,182.330002,0.040971,1.192651e+07,2941.061638,161.100006,1.141906e+07,0.957453,0.883563,1260.0,-1420.0,0.017774,1.142046e+07,1548.694088,0.017781


In [13]:
mayo = dt.date(2020,5,1)
comienzo = mayo - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, mayo)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
mayo = copy.copy(portfolio)
portfolio

[*********************100%***********************]  10 of 10 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
AAPL,271.0,254.300003,0.006034,1.142046e+07,1548.694088,294.470001,1.311285e+07,1.14819,1.157963,270.0,-1.0,0.006063,1.311245e+07,1955.6898,0.006065
ADBE,5812.0,318.390015,0.162032,1.142046e+07,1548.694088,353.640015,1.311285e+07,1.14819,1.110713,6008.0,196.0,0.162035,1.311245e+07,1955.6898,0.162016
AMD,40728.0,45.439999,0.162050,1.142046e+07,1548.694088,52.330002,1.311285e+07,1.14819,1.151629,40602.0,-126.0,0.162037,1.311245e+07,1955.6898,0.162016
AMZN,949.0,1949.790039,0.162021,1.142046e+07,1548.694088,2469.000000,1.311285e+07,1.14819,1.266290,860.0,-89.0,0.161933,1.311245e+07,1955.6898,0.162016
BA,351.0,149.119995,0.004583,1.142046e+07,1548.694088,141.199905,1.311285e+07,1.14819,0.946888,426.0,75.0,0.004587,1.311245e+07,1955.6898,0.004590
MA,4749.0,241.539993,0.100440,1.142046e+07,1548.694088,275.720001,1.311285e+07,1.14819,1.141509,4777.0,28.0,0.100448,1.311245e+07,1955.6898,0.100436
MSFT,6614.0,157.639999,0.091295,1.142046e+07,1548.694088,179.145004,1.311285e+07,1.14819,1.136418,6684.0,70.0,0.091318,1.311245e+07,1955.6898,0.091309
NFLX,4006.0,375.549988,0.131733,1.142046e+07,1548.694088,420.390015,1.311285e+07,1.14819,1.119398,4110.0,104.0,0.131768,1.311245e+07,1955.6898,0.131752
NVDA,7020.0,263.609985,0.162037,1.142046e+07,1548.694088,292.049988,1.311285e+07,1.14819,1.107887,7275.0,255.0,0.162034,1.311245e+07,1955.6898,0.162016
V,1260.0,161.100006,0.017774,1.142046e+07,1548.694088,178.750000,1.311285e+07,1.14819,1.109559,1304.0,44.0,0.017776,1.311245e+07,1955.6898,0.017784


In [14]:
junio = dt.date(2020,6,1)
comienzo = junio - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, junio)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
junio = copy.copy(portfolio)
portfolio

[*********************100%***********************]  10 of 10 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
AAPL,270.0,294.470001,0.006063,1.311245e+07,1955.6898,319.269989,1.398338e+07,1.06642,1.084219,15.0,-255.0,0.000342,1.398332e+07,2017.756817,0.000352
ADBE,6008.0,353.640015,0.162035,1.311245e+07,1955.6898,386.911011,1.398338e+07,1.06642,1.094082,5916.0,-92.0,0.163693,1.398332e+07,2017.756817,0.163678
AMD,40602.0,52.330002,0.162037,1.311245e+07,1955.6898,53.669998,1.398338e+07,1.06642,1.025607,42651.0,2049.0,0.163701,1.398332e+07,2017.756817,0.163678
AMZN,860.0,2469.000000,0.161933,1.311245e+07,1955.6898,2439.659912,1.398338e+07,1.06642,0.988117,938.0,78.0,0.163652,1.398332e+07,2017.756817,0.163678
BA,426.0,141.199905,0.004587,1.311245e+07,1955.6898,145.949997,1.398338e+07,1.06642,1.033641,19.0,-407.0,0.000198,1.398332e+07,2017.756817,0.000203
MA,4777.0,275.720001,0.100448,1.311245e+07,1955.6898,301.589996,1.398338e+07,1.06642,1.093827,4519.0,-258.0,0.097465,1.398332e+07,2017.756817,0.097472
MSFT,6684.0,179.145004,0.091318,1.311245e+07,1955.6898,184.080002,1.398338e+07,1.06642,1.027548,6118.0,-566.0,0.080539,1.398332e+07,2017.756817,0.080530
NFLX,4110.0,420.390015,0.131768,1.311245e+07,1955.6898,420.190002,1.398338e+07,1.06642,0.999524,5447.0,1337.0,0.163679,1.398332e+07,2017.756817,0.163678
NVDA,7275.0,292.049988,0.162034,1.311245e+07,1955.6898,354.630005,1.398338e+07,1.06642,1.214278,6454.0,-821.0,0.163679,1.398332e+07,2017.756817,0.163678
V,1304.0,178.750000,0.017776,1.311245e+07,1955.6898,195.725006,1.398338e+07,1.06642,1.094965,218.0,-1086.0,0.003051,1.398332e+07,2017.756817,0.003052


In [15]:
julio = dt.date(2020,7,1)
comienzo = julio - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, julio)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
julio = copy.copy(portfolio)
portfolio

[*********************100%***********************]  10 of 10 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
AAPL,15.0,319.269989,0.000342,1.398332e+07,2017.756817,364.549988,1.496039e+07,1.069874,1.141824,14.0,-1.0,0.000341,1.495935e+07,3057.291866,0.000355
ADBE,5916.0,386.911011,0.163693,1.398332e+07,2017.756817,435.290009,1.496039e+07,1.069874,1.125039,5625.0,-291.0,0.163677,1.495935e+07,3057.291866,0.163665
AMD,42651.0,53.669998,0.163701,1.398332e+07,2017.756817,52.580002,1.496039e+07,1.069874,0.979691,46573.0,3922.0,0.163697,1.495935e+07,3057.291866,0.163665
AMZN,938.0,2439.659912,0.163652,1.398332e+07,2017.756817,2756.090088,1.496039e+07,1.069874,1.129703,888.0,-50.0,0.163604,1.495935e+07,3057.291866,0.163665
BA,19.0,145.949997,0.000198,1.398332e+07,2017.756817,183.070007,1.496039e+07,1.069874,1.254334,16.0,-3.0,0.000196,1.495935e+07,3057.291866,0.000203
MA,4519.0,301.589996,0.097465,1.398332e+07,2017.756817,295.559998,1.496039e+07,1.069874,0.980006,4935.0,416.0,0.097503,1.495935e+07,3057.291866,0.097490
MSFT,6118.0,184.080002,0.080539,1.398332e+07,2017.756817,203.449997,1.496039e+07,1.069874,1.105226,5925.0,-193.0,0.080581,1.495935e+07,3057.291866,0.080574
NFLX,5447.0,420.190002,0.163679,1.398332e+07,2017.756817,454.769989,1.496039e+07,1.069874,1.082296,5384.0,-63.0,0.163676,1.495935e+07,3057.291866,0.163665
NVDA,6454.0,354.630005,0.163679,1.398332e+07,2017.756817,379.445496,1.496039e+07,1.069874,1.069976,6453.0,-1.0,0.163681,1.495935e+07,3057.291866,0.163665
V,218.0,195.725006,0.003051,1.398332e+07,2017.756817,192.899994,1.496039e+07,1.069874,0.985566,236.0,18.0,0.003043,1.495935e+07,3057.291866,0.003053


In [16]:
agosto = dt.date(2020,8,1)
comienzo = agosto - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, agosto)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
agosto = copy.copy(portfolio)
portfolio

[*********************100%***********************]  10 of 10 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
AAPL,14.0,364.549988,0.000341,1.495935e+07,3057.291866,425.179993,1.708141e+07,1.141855,1.166315,0.0,-14.0,0.000000,1.708106e+07,3406.548107,1.106899e-06
ADBE,5625.0,435.290009,0.163677,1.495935e+07,3057.291866,444.410004,1.708141e+07,1.141855,1.020952,6528.0,903.0,0.169844,1.708106e+07,3406.548107,1.698105e-01
AMD,46573.0,52.580002,0.163697,1.495935e+07,3057.291866,77.430000,1.708141e+07,1.141855,1.472613,37467.0,-9106.0,0.169841,1.708106e+07,3406.548107,1.698105e-01
AMZN,888.0,2756.090088,0.163604,1.495935e+07,3057.291866,3164.679932,1.708141e+07,1.141855,1.148250,916.0,28.0,0.169711,1.708106e+07,3406.548107,1.698105e-01
BA,16.0,183.070007,0.000196,1.495935e+07,3057.291866,158.009995,1.708141e+07,1.141855,0.863112,0.0,-16.0,0.000000,1.708106e+07,3406.548107,3.711289e-07
MA,4935.0,295.559998,0.097503,1.495935e+07,3057.291866,308.519989,1.708141e+07,1.141855,1.043849,4964.0,29.0,0.089660,1.708106e+07,3406.548107,8.964538e-02
MSFT,5925.0,203.449997,0.080581,1.495935e+07,3057.291866,205.100006,1.708141e+07,1.141855,1.008110,5098.0,-827.0,0.061214,1.708106e+07,3406.548107,6.121270e-02
NFLX,5384.0,454.769989,0.163676,1.495935e+07,3057.291866,489.269989,1.708141e+07,1.141855,1.075863,5929.0,545.0,0.169830,1.708106e+07,3406.548107,1.698105e-01
NVDA,6453.0,379.445496,0.163681,1.495935e+07,3057.291866,424.579987,1.708141e+07,1.141855,1.118949,6832.0,379.0,0.169821,1.708106e+07,3406.548107,1.698105e-01
V,236.0,192.899994,0.003043,1.495935e+07,3057.291866,190.399994,1.708141e+07,1.141855,0.987040,7.0,-229.0,0.000078,1.708106e+07,3406.548107,8.786481e-05


In [17]:
septiembre = dt.date(2020,9,1)
comienzo = septiembre - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, septiembre)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
septiembre = copy.copy(portfolio)
portfolio

[*********************100%***********************]  10 of 10 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
AAPL,0.0,425.179993,0.000000,1.708106e+07,3406.548107,128.850006,1.962347e+07,1.148843,0.303048,0.0,0.0,0.000000,1.962484e+07,2033.407589,1.098339e-06
ADBE,6528.0,444.410004,0.169844,1.708106e+07,3406.548107,513.390015,1.962347e+07,1.148843,1.155217,6490.0,-38.0,0.169780,1.962484e+07,2033.407589,1.697863e-01
AMD,37467.0,77.430000,0.169841,1.708106e+07,3406.548107,90.800003,1.962347e+07,1.148843,1.172672,36700.0,-767.0,0.169803,1.962484e+07,2033.407589,1.697863e-01
AMZN,916.0,3164.679932,0.169711,1.708106e+07,3406.548107,3441.909912,1.962347e+07,1.148843,1.087601,968.0,52.0,0.169773,1.962484e+07,2033.407589,1.697863e-01
BA,0.0,158.009995,0.000000,1.708106e+07,3406.548107,171.830002,1.962347e+07,1.148843,1.087463,0.0,0.0,0.000000,1.962484e+07,2033.407589,3.617767e-07
MA,4964.0,308.519989,0.089660,1.708106e+07,3406.548107,358.000000,1.962347e+07,1.148843,1.160379,4918.0,-46.0,0.089715,1.962484e+07,2033.407589,8.970813e-02
MSFT,5098.0,205.100006,0.061214,1.708106e+07,3406.548107,225.059998,1.962347e+07,1.148843,1.097318,5343.0,245.0,0.061274,1.962484e+07,2033.407589,6.127147e-02
NFLX,5929.0,489.269989,0.169830,1.708106e+07,3406.548107,529.049988,1.962347e+07,1.148843,1.081305,6298.0,369.0,0.169783,1.962484e+07,2033.407589,1.697863e-01
NVDA,6832.0,424.579987,0.169821,1.708106e+07,3406.548107,534.920227,1.962347e+07,1.148843,1.259881,6229.0,-603.0,0.169786,1.962484e+07,2033.407589,1.697863e-01
V,7.0,190.399994,0.000078,1.708106e+07,3406.548107,212.000000,1.962347e+07,1.148843,1.113445,8.0,1.0,0.000086,1.962484e+07,2033.407589,8.738857e-05


In [18]:
octubre = dt.date(2020,10,1)
comienzo = octubre - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, octubre)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
octubre = copy.copy(portfolio)
portfolio

[*********************100%***********************]  10 of 10 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
AAPL,0.0,128.850006,0.000000,1.962484e+07,2033.407589,115.610001,1.853746e+07,0.944592,0.897245,0.0,0.0,0.000000,1.853765e+07,1837.823803,0.000000e+00
ADBE,6490.0,513.390015,0.169780,1.962484e+07,2033.407589,489.739990,1.853746e+07,0.944592,0.953934,6724.0,234.0,0.177639,1.853765e+07,1837.823803,1.776277e-01
AMD,36700.0,90.800003,0.169803,1.962484e+07,2033.407589,81.957397,1.853746e+07,0.944592,0.902614,40180.0,3480.0,0.177641,1.853765e+07,1837.823803,1.776277e-01
AMZN,968.0,3441.909912,0.169773,1.962484e+07,2033.407589,3147.425049,1.853746e+07,0.944592,0.914441,1046.0,78.0,0.177596,1.853765e+07,1837.823803,1.776277e-01
BA,0.0,171.830002,0.000000,1.962484e+07,2033.407589,165.144394,1.853746e+07,0.944592,0.961092,0.0,0.0,0.000000,1.853765e+07,1837.823803,0.000000e+00
MA,4918.0,358.000000,0.089715,1.962484e+07,2033.407589,337.649994,1.853746e+07,0.944592,0.943156,4189.0,-729.0,0.076300,1.853765e+07,1837.823803,7.629737e-02
MSFT,5343.0,225.059998,0.061274,1.962484e+07,2033.407589,210.330002,1.853746e+07,0.944592,0.934551,3134.0,-2209.0,0.035559,1.853765e+07,1837.823803,3.556398e-02
NFLX,6298.0,529.049988,0.169783,1.962484e+07,2033.407589,500.049988,1.853746e+07,0.944592,0.945185,6585.0,287.0,0.177629,1.853765e+07,1837.823803,1.776277e-01
NVDA,6229.0,534.920227,0.169786,1.962484e+07,2033.407589,540.895020,1.853746e+07,0.944592,1.011170,6088.0,-141.0,0.177637,1.853765e+07,1837.823803,1.776277e-01
V,8.0,212.000000,0.000086,1.962484e+07,2033.407589,199.600006,1.853746e+07,0.944592,0.941509,0.0,-8.0,0.000000,1.853765e+07,1837.823803,5.778423e-08


In [19]:
noviembre = dt.date(2020,11,1)
comienzo = noviembre - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, noviembre)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
noviembre = copy.copy(portfolio)
portfolio

[*********************100%***********************]  10 of 10 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
AAPL,0.0,115.610001,0.000000,1.853765e+07,1837.823803,108.945000,1.724500e+07,0.930269,0.942349,0.0,0.0,0.000000,1.724277e+07,4068.679028,0.000000e+00
ADBE,6724.0,489.739990,0.177639,1.853765e+07,1837.823803,447.100006,1.724500e+07,0.930269,0.912933,6850.0,126.0,0.177619,1.724277e+07,4068.679028,1.775956e-01
AMD,40180.0,81.957397,0.177641,1.853765e+07,1837.823803,75.300003,1.724500e+07,0.930269,0.918770,40676.0,496.0,0.177634,1.724277e+07,4068.679028,1.775956e-01
AMZN,1046.0,3147.425049,0.177596,1.853765e+07,1837.823803,3036.129883,1.724500e+07,0.930269,0.964639,1008.0,-38.0,0.177490,1.724277e+07,4068.679028,1.775956e-01
BA,0.0,165.144394,0.000000,1.853765e+07,1837.823803,144.389999,1.724500e+07,0.930269,0.874326,0.0,0.0,0.000000,1.724277e+07,4068.679028,0.000000e+00
MA,4189.0,337.649994,0.076300,1.853765e+07,1837.823803,289.119995,1.724500e+07,0.930269,0.856271,4556.0,367.0,0.076393,1.724277e+07,4068.679028,7.638890e-02
MSFT,3134.0,210.330002,0.035559,1.853765e+07,1837.823803,202.520004,1.724500e+07,0.930269,0.962868,3034.0,-100.0,0.035635,1.724277e+07,4068.679028,3.563314e-02
NFLX,6585.0,500.049988,0.177629,1.853765e+07,1837.823803,475.779999,1.724500e+07,0.930269,0.951465,6437.0,-148.0,0.177616,1.724277e+07,4068.679028,1.775956e-01
NVDA,6088.0,540.895020,0.177637,1.853765e+07,1837.823803,502.385010,1.724500e+07,0.930269,0.928803,6096.0,8.0,0.177613,1.724277e+07,4068.679028,1.775956e-01
V,0.0,199.600006,0.000000,1.853765e+07,1837.823803,181.830002,1.724500e+07,0.930269,0.910972,0.0,0.0,0.000000,1.724277e+07,4068.679028,7.088247e-08


In [20]:
diciembre = dt.date(2020,12,1)
comienzo = diciembre - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, diciembre)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
diciembre = copy.copy(portfolio)
portfolio

[*********************100%***********************]  10 of 10 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
AAPL,0.0,108.945000,0.000000,1.724277e+07,4068.679028,119.059998,1.885538e+07,1.093524,1.092845,0.0,0.0,0.000000,1.885519e+07,4251.048626,0.000000
ADBE,6850.0,447.100006,0.177619,1.724277e+07,4068.679028,478.595001,1.885538e+07,1.093524,1.070443,7354.0,504.0,0.186664,1.885519e+07,4251.048626,0.186629
AMD,40676.0,75.300003,0.177634,1.724277e+07,4068.679028,92.709999,1.885538e+07,1.093524,1.231208,37964.0,-2712.0,0.186667,1.885519e+07,4251.048626,0.186629
AMZN,1008.0,3036.129883,0.177490,1.724277e+07,4068.679028,3168.209961,1.885538e+07,1.093524,1.043503,1110.0,102.0,0.186512,1.885519e+07,4251.048626,0.186629
BA,0.0,144.389999,0.000000,1.724277e+07,4068.679028,210.937195,1.885538e+07,1.093524,1.460885,0.0,0.0,0.000000,1.885519e+07,4251.048626,0.000000
MA,4556.0,289.119995,0.076393,1.724277e+07,4068.679028,336.589996,1.885538e+07,1.093524,1.164188,3077.0,-1479.0,0.054928,1.885519e+07,4251.048626,0.054925
MSFT,3034.0,202.520004,0.035635,1.724277e+07,4068.679028,214.309998,1.885538e+07,1.093524,1.058216,1049.0,-1985.0,0.011923,1.885519e+07,4251.048626,0.011929
NFLX,6437.0,475.779999,0.177616,1.724277e+07,4068.679028,490.700012,1.885538e+07,1.093524,1.031359,7172.0,735.0,0.186649,1.885519e+07,4251.048626,0.186629
NVDA,6096.0,502.385010,0.177613,1.724277e+07,4068.679028,536.419983,1.885538e+07,1.093524,1.067747,6561.0,465.0,0.186657,1.885519e+07,4251.048626,0.186629
V,0.0,181.830002,0.000000,1.724277e+07,4068.679028,210.539993,1.885538e+07,1.093524,1.157895,0.0,0.0,0.000000,1.885519e+07,4251.048626,0.000000


In [21]:
fin = dt.date(2020,12,31)
comienzo = fin - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, fin)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
fin = copy.copy(portfolio)
portfolio

[*********************100%***********************]  10 of 10 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
AAPL,0.0,119.059998,0.000000,1.885519e+07,4251.048626,133.679993,1.934223e+07,1.02583,1.122795,0.0,0.0,0.000000,1.934367e+07,2812.255024,0.000000
ADBE,7354.0,478.595001,0.186664,1.885519e+07,4251.048626,497.049988,1.934223e+07,1.02583,1.038561,7262.0,-92.0,0.186602,1.934367e+07,2812.255024,0.186591
AMD,37964.0,92.709999,0.186667,1.885519e+07,4251.048626,92.269997,1.934223e+07,1.02583,0.995254,39123.0,1159.0,0.186618,1.934367e+07,2812.255024,0.186591
AMZN,1110.0,3168.209961,0.186512,1.885519e+07,4251.048626,3285.850098,1.934223e+07,1.02583,1.037131,1098.0,-12.0,0.186514,1.934367e+07,2812.255024,0.186591
BA,0.0,210.937195,0.000000,1.885519e+07,4251.048626,216.598297,1.934223e+07,1.02583,1.026838,0.0,0.0,0.000000,1.934367e+07,2812.255024,0.000000
MA,3077.0,336.589996,0.054928,1.885519e+07,4251.048626,355.600006,1.934223e+07,1.02583,1.056478,2995.0,-82.0,0.055058,1.934367e+07,2812.255024,0.055062
MSFT,1049.0,214.309998,0.011923,1.885519e+07,4251.048626,221.669998,1.934223e+07,1.02583,1.034343,1045.0,-4.0,0.011975,1.934367e+07,2812.255024,0.011981
NFLX,7172.0,490.700012,0.186649,1.885519e+07,4251.048626,524.609985,1.934223e+07,1.02583,1.069105,6881.0,-291.0,0.186616,1.934367e+07,2812.255024,0.186591
NVDA,6561.0,536.419983,0.186657,1.885519e+07,4251.048626,525.450012,1.934223e+07,1.02583,0.979550,6870.0,309.0,0.186616,1.934367e+07,2812.255024,0.186591
V,0.0,210.539993,0.000000,1.885519e+07,4251.048626,218.350006,1.934223e+07,1.02583,1.037095,0.0,0.0,0.000000,1.934367e+07,2812.255024,0.000000


# $ 1000000 Inicio
# $ 1934700 Final
# Ganancia del 93,47%

# Opcion Estrategia Pasiva, no rebalanceo en todo 1 año

# Cartera Rabalanceo 66,72% (1646 / 987) 
# Cartera Balance Constante 57,68%
# Cartera Buy-&-Hold 58,06%

In [22]:
writer = pd.ExcelWriter('riskRebalance.xlsx', engine='xlsxwriter')
enero.to_excel(writer,sheet_name='enero')
febrero.to_excel(writer,sheet_name='febrero')
marzo.to_excel(writer,sheet_name='marzo')
abril.to_excel(writer,sheet_name='abril')
mayo.to_excel(writer,sheet_name='mayo')
junio.to_excel(writer,sheet_name='junio')
julio.to_excel(writer,sheet_name='julio')
agosto.to_excel(writer,sheet_name='agosto')
septiembre.to_excel(writer,sheet_name='septiembre')
octubre.to_excel(writer,sheet_name='octubre')
noviembre.to_excel(writer,sheet_name='noviembre')
diciembre.to_excel(writer,sheet_name='diciembre')
fin.to_excel(writer,sheet_name='fin')
writer.save()